## Проект по SQL

#### Цель проекта: изучить данные о книгах из разных таблиц и вывести необходимые данные

#### Подключимся к базе данных:

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Выгрузим данные из базы

In [2]:
books = pd.io.sql.read_sql('books', con = engine)
authors = pd.io.sql.read_sql('authors', con = engine)
ratings = pd.io.sql.read_sql('ratings', con = engine)
reviews = pd.io.sql.read_sql('reviews', con = engine)
publishers = pd.io.sql.read_sql('publishers', con = engine)

#### Исследуем таблицы и выведем первые строки:


In [3]:
def data_start(data):
    display(data.head())
    data.info()
datas =[books, authors, ratings, reviews, publishers]    
for data in datas:
    data_start(data)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
book_id             1000 non-null int64
author_id           1000 non-null int64
title               1000 non-null object
num_pages           1000 non-null int64
publication_date    1000 non-null datetime64[ns]
publisher_id        1000 non-null int64
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 47.0+ KB


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
author_id    636 non-null int64
author       636 non-null object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
rating_id    6456 non-null int64
book_id      6456 non-null int64
username     6456 non-null object
rating       6456 non-null int64
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
review_id    2793 non-null int64
book_id      2793 non-null int64
username     2793 non-null object
text         2793 non-null object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
publisher_id    340 non-null int64
publisher       340 non-null object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


#### Выполним задания

Посчитаем, сколько книг вышло после 1 января 2000 года

In [4]:
pd.io.sql.read_sql("""SELECT
count(book_id)
FROM
books 
WHERE
publication_date > '2000-01-01'""", con = engine)

,count
0,819


Для каждой книги посчитаем количество обзоров и среднюю оценку

In [5]:
pd.io.sql.read_sql("""SELECT
books.book_id, count(DISTINCT review_id) as reviews, avg(rating) as mean_rating 
FROM
books LEFT JOIN ratings on ratings.book_id = books.book_id LEFT JOIN reviews on reviews.book_id = ratings.book_id 
GROUP BY
books.book_id""", con = engine)

,book_id,reviews,mean_rating
0,1,2,3.666667
1,2,1,2.500000
2,3,3,4.666667
3,4,2,4.500000
4,5,4,4.000000
...,...,...,...
995,996,3,3.666667
996,997,3,3.400000
997,998,4,3.200000
998,999,2,4.500000


Определим издательство, которое выпустило наибольшее число книг толще 50 страниц.


In [6]:
pd.io.sql.read_sql("""SELECT
publishers.publisher, count(books.book_id) as books 
FROM
books LEFT JOIN publishers on publishers.publisher_id = books.publisher_id 
WHERE
num_pages > 50 GROUP BY publishers.publisher 
ORDER BY
count(books.book_id) DESC 
LIMIT 1""", con = engine)

,publisher,books
0,Penguin Books,42


Определим автора с самой высокой средней оценкой книг (учитываем книги более чем с 50 оценками)

In [7]:
pd.io.sql.read_sql("""SELECT
authors.author, books.author_id, AVG(ratings.rating) as mean_rating 
FROM
(SELECT
book_id, COUNT(rating_id) as ratings_cnt 
FROM
ratings 
GROUP BY book_id 
HAVING
COUNT(DISTINCT rating_id) > 50) as SUBQ LEFT JOIN books on books.book_id = SUBQ.book_id LEFT JOIN ratings on ratings.book_id = SUBQ.book_id LEFT JOIN authors on authors.author_id = books.author_id
GROUP BY
authors.author, books.author_id
ORDER BY
mean_rating DESC
LIMIT 1""", con = engine)

,author,author_id,mean_rating
0,J.K. Rowling/Mary GrandPré,236,4.287097


Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [8]:
pd.io.sql.read_sql("""SELECT
AVG(SUBQ.reviews) 
FROM
(SELECT
reviews.username, COUNT(DISTINCT reviews.review_id) as reviews, COUNT(DISTINCT ratings.rating_id) as ratings
FROM
reviews LEFT JOIN ratings on ratings.username = reviews.username
GROUP BY
reviews.username 
HAVING
COUNT(DISTINCT ratings.rating_id) > 50) as SUBQ""", con = engine)

,avg
0,24.333333
